In [84]:
import json
import pandas as pd

In [85]:
df = pd.read_csv('data/kodas.csv')

print (len(df))

9588


In [86]:
attribute_dict = {}

for i, row in df.iterrows():
    
    for trait_type in df.columns[2:]:

        trait_value = row[trait_type]

        if trait_type not in attribute_dict:
            attribute_dict[trait_type] = {}
        
        if trait_value not in attribute_dict[trait_type]:
            attribute_dict[trait_type][trait_value] = 1
        else:
            attribute_dict[trait_type][trait_value] += 1

with open('data/kodas_attribute_count.json', 'w') as f:
    json.dump(attribute_dict, f)            
    

In [87]:
df.isnull().describe()

,token_id,image,head,eyes,core,clothing,weapon
count,9588,9588,9588,9588,9588,9588,9588
unique,1,1,1,1,1,2,2
top,False,False,False,False,False,False,True
freq,9588,9588,9588,9588,9588,6766,6558


In [88]:
df.shape[0]

9588

In [89]:
for trait_type in attribute_dict.keys():

    trait_null_count = df[trait_type].isnull().describe().freq / df.shape[0]
    print (trait_type, trait_null_count)

    df['{}_count'.format(trait_type)]  = [trait_null_count if pd.isnull(x) else attribute_dict[trait_type][x] for x in df[trait_type]]   
    df['{}_rarity'.format(trait_type)] = [trait_null_count / df.shape[0]if pd.isnull(x) else attribute_dict[trait_type][x] / df.shape[0] for x in df[trait_type]]    

head 1.0
eyes 1.0
core 1.0
clothing 0.7056737588652482
weapon 0.6839799749687109


In [90]:
df['rarity'] = df['head_rarity'] * df['eyes_rarity'] * df['core_rarity'] * df['clothing_rarity'] * df['weapon_rarity']

In [91]:
df['rarity_score'] = [df.query('rarity < {}'.format(x)).count()[0] for x in df['rarity']]

In [92]:
df.sort_values('rarity', ascending=True).head()

,token_id,image,head,eyes,core,clothing,weapon,head_count,head_rarity,eyes_count,eyes_rarity,core_count,core_rarity,clothing_count,clothing_rarity,weapon_count,weapon_rarity,rarity,rarity_score
9546,9996,https://assets.otherside.xyz/kodas/059b92470fd...,Cyber Mind,Coiled Up,Dyna Volt,Chaos Gunner,Cosmic Hammer,1,0.000104,1,0.000104,1,0.000104,1.000000,0.000104,1.0,0.000104,1.234127e-20,0
9545,9995,https://assets.otherside.xyz/kodas/511a0bc2bdf...,Mech Mouse,Galactic Gaze,Static Enigma,Chaos Soldier,Truestrike,1,0.000104,1,0.000104,1,0.000104,1.000000,0.000104,1.0,0.000104,1.234127e-20,0
9544,9994,https://assets.otherside.xyz/kodas/a5caa4c5eb3...,Tazor,Light Visor,Kilowatt,Chaos Avenger,Coils of Vengeance,1,0.000104,1,0.000104,1,0.000104,1.000000,0.000104,1.0,0.000104,1.234127e-20,0
9491,9938,https://assets.otherside.xyz/kodas/34322ce6b43...,Turitella Dome,Aquatic Ogle,Gilly Guts,NaN,Urchin Flail,1,0.000104,21,0.002190,1,0.000104,0.705674,0.000074,18.0,0.001877,3.291968e-18,3
9466,9912,https://assets.otherside.xyz/kodas/1c2fb82e20b...,Hot Head,Interconnected Flow,Bellystone,Flame Licked Armor,Sulfur Striker,1,0.000104,23,0.002399,1,0.000104,18.000000,0.001877,3.0,0.000313,1.532786e-17,4


In [93]:
df.to_csv('data/kodas_with_rarity.csv', index=False)